In [ ]:
# Parcial: Curso Análisis Predictivo de Series Temporales

## Posgrado de Analítica de Big Data

## Universidad ORT Uruguay

### Curso 2020

**Fecha límite de entrega:** 17/7/2020, 21:00hs. por el [sitio de gestion](http://gestion.ort.edu.uy).

In [ ]:
## Ejercicio

El ejercicio consiste en analizar diferentes modelos para la serie de muertes (cada 10.000 habitantes) relacionadas con influenza y neumonia en EEUU, disponible en la biblioteca `astsa`. La serie, denominada `flu`, comienza en 1968 y toma valores mensuales hasta 1978 inclusive. 

Se propone:

1. Realizar un análisis exploratorio de la serie, determinando tendencias y posibles estacionalidades. Analizar también la autocorrelación de la serie.

2. Realizar un ajuste lineal basado en tendencia y factores estacionales (en principio uno por mes).

3. Realizar un ajuste lineal basado en tendencia y frecuencias. Justificar la elección de las frecuencias incluidas en el ajuste.

4. Comparar los ajustes anteriores desde el punto de vista del error cometido. ¿Con cuál se quedaría si debiera elegir? Justifique.

5. A los residuos de ambos ajustes anteriores, analizar si es posible modelarlos mediante un proceso ARMA(p,q) adecuado. Justifique la elección del orden en cada caso y analice los residuos obtenidos (no importa si no son Gaussianos).

6. Discutir cuál de los caminos seguidos logra mejor ajuste luego de ajustar los residuos.

7. Realizar una predicción completa para los años 1979 y 1980 de la serie a partir del ajuste que crea conveniente.

8. Se dispone además de una serie de temperaturas medias mensuales en EEUU en el archivo `temperaturas.csv`. Analizar:
 * Si hay correlación cruzada entre la serie de muertes por gripe y neumonia y la temperatura observada en cada mes.
 
 * Si esta correlación se mantiene una vez que se saca el efecto estacional a la temperatura.
 
 
**Nota:** Para el análisis pueden utilizarse todas las bibliotecas vistas en el curso así como otras que entiendan convenientes. Se recomienda en particular `forecast` para algunos ajustes y predicciones. 

In [38]:
install.packages("astsa")
install.packages("forecast")
library(lmtest)
library(forecast)
library(astsa)
tsplot(flu)
options(repr.plot.width=10, repr.plot.height=6) #ajusta tamaño de graficas

Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric



In [4]:
flu

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
1968,0.8113721,0.4458291,0.3415985,0.2774243,0.2484958,0.2525427,0.2466902,0.2452006,0.2279679,0.2610293,0.3177998,0.7298681
1969,0.8192756,0.4376872,0.3834813,0.2919304,0.2556420,0.2369918,0.2495123,0.2280816,0.2335083,0.2679917,0.3030445,0.3594536
1970,0.4932865,0.5692177,0.3593959,0.2741868,0.2424912,0.2241473,0.2280840,0.2283657,0.2282342,0.2579092,0.2909701,0.3113483
1971,0.3743171,0.3746391,0.3398281,0.2909505,0.2403667,0.2269297,0.2134564,0.2074597,0.2099793,0.2464345,0.2768291,0.3359459
1972,0.6381312,0.5195216,0.2968231,0.2514277,0.2144966,0.1964111,0.2343862,0.2095384,0.2177600,0.2450697,0.2554203,0.3357194
1973,0.5968794,0.5156948,0.3085772,0.2569596,0.2194814,0.2158074,0.2240929,0.2217431,0.2388084,0.2528328,0.2605210,0.2822165
1974,0.3096678,0.3330646,0.3497020,0.3066515,0.2238370,0.2030033,0.2231299,0.1891271,0.1997522,0.2269953,0.2295186,0.3198083
1975,0.4937200,0.4728154,0.3094317,0.2343683,0.2007220,0.1906996,0.1877136,0.1990874,0.1908602,0.2111834,0.2084524,0.2510464
1976,0.2902977,0.5702096,0.6384278,0.3135166,0.2047724,0.1885323,0.1798065,0.1867087,0.1871117,0.2157304,0.2385934,0.2572363
1977,0.2817310,0.3030367,0.2915253,0.2453148,0.2051037,0.1845583,0.2046436,0.1828399,0.1960763,0.2203844,0.2249940,0.3036654


### $Parte 1:$ 
Realizar un análisis exploratorio de la serie, determinando tendencias y posibles estacionalidades. Analizar también la autocorrelación de la serie.

In [5]:
plot(decompose(flu)) #Analizamos la descomposicion de la serie

In [11]:
#Le realizamos el periodigrama a los residuos del Fit_0 para sacarle la tendencia.

Fit_0 = lm(flu~t)

n<-length(residuals(Fit_0))
I = abs(fft(residuals(Fit_0)))^2/n 
P = (4/n)*I[1:(n/2)]
f = 12 * (0:(n/2-1))/n

plot(f, P, type="l", col = 8, xlab="Frequency", ylab="Scaled Periodogram")
tsplot(f,P, xlim=c(0,10))

𝑂𝑏𝑠𝑒𝑟𝑣𝑎𝑐𝑖ó𝑛:  Por medio del periodograma podemos observar que existe un componente periodico, según la gráfica anterior se observa claramente un peirodo que se repite de forma anual, lo que indica que hay un ciclo periodico cada año. También se puede observar que existe pero mucho mas leve tambien internamente dos ciclos anuales.
En el decompose de mas arriba se puede observar una leve tendencia decreciente de la serie.

### $Parte2:$
Realizar un ajuste lineal basado en tendencia y factores estacionales (en principio uno por mes).


In [7]:
season = factor(cycle(flu)) 
t = time(flu)-1968

Fit_1 = lm(flu ~ 0 + t + season )
summary(Fit_1)


Call:
lm(formula = flu ~ 0 + t + season)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.19788 -0.02061 -0.00515  0.01230  0.34261 

Coefficients:
          Estimate Std. Error t value Pr(>|t|)    
t        -0.009393   0.002052  -4.578 1.16e-05 ***
season1   0.563324   0.024707  22.800  < 2e-16 ***
season2   0.500194   0.024778  20.187  < 2e-16 ***
season3   0.403202   0.024851  16.225  < 2e-16 ***
season4   0.320307   0.024924  12.851  < 2e-16 ***
season5   0.275098   0.024999  11.005  < 2e-16 ***
season6   0.260889   0.025074  10.405  < 2e-16 ***
season7   0.268409   0.025150  10.672  < 2e-16 ***
season8   0.260038   0.025227  10.308  < 2e-16 ***
season9   0.264541   0.025306  10.454  < 2e-16 ***
season10  0.293523   0.025385  11.563  < 2e-16 ***
season11  0.312202   0.025464  12.260  < 2e-16 ***
season12  0.395871   0.025545  15.497  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.07455 on 119 degrees of freedo

In [8]:
acf2(residuals(Fit_1), , col = 8)

ACF,0.36,-0.13,-0.12,-0.08,-0.04,0.00,0.00,-0.04,-0.07,-0.16,...,0.05,0.12,0.02,-0.02,-0.01,-0.02,-0.02,0.00,0.01,0.11
PACF,0.36,-0.29,0.06,-0.10,0.02,-0.02,-0.01,-0.05,-0.05,-0.15,...,0.05,0.09,-0.09,0.08,-0.05,0.00,-0.01,-0.02,0.02,0.14


In [9]:
Ajuste_1 = ts(fitted(Fit_1),start=1968,freq=12)

tsplot(flu)
lines(Ajuste_1,col=12)

### $Parte 3:$
Realizar un ajuste lineal basado en tendencia y frecuencias. Justificar la elección de las frecuencias incluidas en el ajuste. 

In [18]:
#Viendo el periodroigrma de la parte 1 utilizamos la frecuencia para un modelo seno y coseno con frecuencia 1 y 2. 
#También comparamos con el modelo de frecuencia 1 solo, y comparamos los modelos segun los residuos.

Fit_2_prueba<-lm(flu~t+sin(2*pi*t)+cos(2*pi*t))

Fit_2<-lm(flu~t+sin(2*pi*t)+cos(2*pi*t)+sin(4*pi*t)+cos(4*pi*t))


SSE_2_prueba = sum(residuals(Fit_2_prueba)^2) #Modelo lineal con frecuencia 1
SSE_2 = sum(residuals(Fit_2)^2) #Modelo lineal con frecuencia 1 y 2


R2 = (SSE_2_prueba-SSE_1)/SSE_2_prueba

#muestro los valores

SSE_2_prueba
SSE_2
R2


# El mejor modelo es que tiene frecuencia 1 y 2 dado que tiene menor SSE 

[1] 0.9401359

[1] 0.7303626

[1] 0.2966043

In [15]:
summary(Fit_2)

acf(residuals(Fit_2), , col = 8)
pacf(residuals(Fit_2),  col = 8)


Call:
lm(formula = flu ~ t + sin(2 * pi * t) + cos(2 * pi * t) + sin(4 * 
    pi * t) + cos(4 * pi * t))

Residuals:
     Min       1Q   Median       3Q      Max 
-0.14469 -0.03035 -0.00558  0.01776  0.31658 

Coefficients:
                 Estimate Std. Error t value Pr(>|t|)    
(Intercept)      0.344655   0.013211  26.089  < 2e-16 ***
t               -0.009672   0.002094  -4.619 9.37e-06 ***
sin(2 * pi * t)  0.027800   0.009394   2.959  0.00368 ** 
cos(2 * pi * t)  0.117923   0.009373  12.581  < 2e-16 ***
sin(4 * pi * t)  0.026505   0.009376   2.827  0.00547 ** 
cos(4 * pi * t)  0.049790   0.009373   5.312 4.75e-07 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.07613 on 126 degrees of freedom
Multiple R-squared:  0.6467,	Adjusted R-squared:  0.6327 
F-statistic: 46.13 on 5 and 126 DF,  p-value: < 2.2e-16


In [16]:
Ajuste_2 = ts(fitted(Fit_2),start=1968,freq=12)

tsplot(flu)
lines(Ajuste_2, col=2)

### $Parte4$:
Comparar los ajustes anteriores desde el punto de vista del error cometido. ¿Con cuál se quedaría si debiera elegir? Justifique.



In [19]:
## Para comparar los ajustes de los modelos y estudiar el error cometido, recurrimos al estimador de suma de cuadrado de los residuos.

SSE = sum((flu-mean(flu))^2) #Este se hace para ver como mejora sacandole la media
SSE_1 = sum(residuals(Fit_1)^2) #Modelo lineal 1
SSE_2 = sum(residuals(Fit_2)^2) #Modelo lineal 2



R2 = (SSE_2-SSE_1)/SSE_2

#muestro los valores
SSE
SSE_1
SSE_2
R2 #El R^2 es una medida de correlación de nuestras variables, o bien cuánto mejora el ajuste en términos relativos respecto a la media.

[1] 2.067407

[1] 0.6612876

[1] 0.7303626

[1] 0.09457639

### Lo que se puede observar que el modelo 1 (Fit_1) en terminos relativos tiene un mejor ajuste del 10% aprox en comparacion con el modelo 2 (Fit_2)

### $Parte 5:$ 
A los residuos de ambos ajustes anteriores, analizar si es posible modelarlos mediante un proceso ARMA(p,q) adecuado. Justifique la elección del orden en cada caso y analice los residuos obtenidos (no importa si no son Gaussianos).



In [20]:
# Modelo lineal 1
y = residuals(Fit_1)
head(y)
acf2(y)


1            2            3            4            5            6 
 0.248047800 -0.053581727 -0.060037655 -0.040534127 -0.023471427 -0.004432509

ACF,0.36,-0.13,-0.12,-0.08,-0.04,0.00,0.00,-0.04,-0.07,-0.16,...,0.05,0.12,0.02,-0.02,-0.01,-0.02,-0.02,0.00,0.01,0.11
PACF,0.36,-0.29,0.06,-0.10,0.02,-0.02,-0.01,-0.05,-0.05,-0.15,...,0.05,0.09,-0.09,0.08,-0.05,0.00,-0.01,-0.02,0.02,0.14


In [41]:
#Según los valors del acf y pacf, le ajustamos un modelo MA(1) y un AR(2)
Fit_1.2 = Arima(y , c(2,0,1),include.mean = FALSE) 
summary(Fit_1.2)

Series: y 
ARIMA(2,0,1) with zero mean 

Coefficients:
         ar1      ar2     ma1
      0.0073  -0.1301  0.6467
s.e.  0.1848   0.1382  0.1683

sigma^2 estimated as 0.003688:  log likelihood=183.64
AIC=-359.29   AICc=-358.97   BIC=-347.75

Training set error measures:
                        ME      RMSE        MAE      MPE     MAPE     MASE
Training set -9.312138e-05 0.0600375 0.03456021 67.14554 174.3773 0.828037
                    ACF1
Training set -0.07959855


In [22]:
acf2(residuals(Fit_1.2))

ACF,-0.08,0.06,-0.08,-0.04,-0.03,-0.01,0.03,-0.08,0.02,-0.14,...,-0.04,0.17,-0.08,0.05,-0.03,0.00,-0.03,0.03,-0.01,0.04
PACF,-0.08,0.05,-0.07,-0.06,-0.03,-0.02,0.02,-0.08,0.00,-0.14,...,-0.06,0.15,-0.06,0.01,0.00,-0.02,-0.02,0.01,-0.02,0.06


In [23]:
# Modelo lineal 2
j = residuals(Fit_2)
head(j)
acf2(j)

1            2            3            4            5            6 
 0.299004359 -0.061893486 -0.082541118 -0.042823201 -0.010200301 -0.001798786

ACF,0.32,-0.17,-0.10,-0.04,-0.04,-0.02,-0.02,-0.01,-0.04,-0.18,...,0.06,0.06,0.01,0.02,0.00,-0.04,-0.04,0.02,0.05,0.08
PACF,0.32,-0.30,0.08,-0.10,0.00,-0.03,-0.02,-0.01,-0.05,-0.19,...,-0.01,0.08,-0.06,0.08,-0.06,-0.01,-0.02,0.00,0.03,0.11


In [43]:
#Según los valors del acf y pacf, le ajustamos un modelo MA(1) y un AR(2)
Fit_2.2 = Arima(j, c(2,0,1),include.mean = FALSE) 
summary(Fit_2.2)

Series: j 
ARIMA(2,0,1) with zero mean 

Coefficients:
         ar1      ar2     ma1
      0.0012  -0.1935  0.6442
s.e.  0.1611   0.1254  0.1434

sigma^2 estimated as 0.00408:  log likelihood=176.94
AIC=-345.87   AICc=-345.56   BIC=-334.34

Training set error measures:
                        ME       RMSE        MAE    MPE     MAPE     MASE
Training set -5.031491e-05 0.06314784 0.04017199 188.49 260.1711 0.797083
                    ACF1
Training set -0.09818008


In [44]:
acf2(residuals(Fit_2.2))

ACF,-0.1,0.08,-0.07,-0.02,-0.05,-0.01,0,-0.06,0.04,-0.15,...,-0.03,0.12,-0.06,0.07,-0.04,0.01,-0.05,0.04,0.01,0.04
PACF,-0.1,0.07,-0.06,-0.04,-0.04,-0.02,0,-0.07,0.02,-0.14,...,-0.03,0.09,-0.05,0.04,-0.01,-0.03,-0.04,0.02,0.01,0.06


### $Parte 6:$

Discutir cuál de los caminos seguidos logra mejor ajuste luego de ajustar los residuos.



In [45]:
## Analizamos los residuos de los modelos

tsplot(residuals(Fit_1.2))
checkresiduals(Fit_1.2)

# El p-valor nos queda mayor que 0.05 por lo que se puede decir que los residuos son ruido blanco


	Ljung-Box test

data:  Residuals from ARIMA(2,0,1) with zero mean
Q* = 6.6123, df = 7, p-value = 0.4703

Model df: 3.   Total lags used: 10



In [46]:
# Ajuste de los residuos del modelo 2

tsplot(residuals(Fit_2.2))
checkresiduals(Fit_2.2)

#En este caso el p-valor quedo por encima del 0,05 y también los residuos son ruido blanco.


	Ljung-Box test

data:  Residuals from ARIMA(2,0,1) with zero mean
Q* = 7.1723, df = 7, p-value = 0.4112

Model df: 3.   Total lags used: 10



In [47]:
AIC(Fit_1.2)
AIC(Fit_2.2)
BIC(Fit_1.2)
BIC(Fit_2.2)

[1] -359.2851

[1] -345.8728

[1] -347.7539

[1] -334.3416

### Como los valores de AIC (criterio de máxima verosimilitud) y BIC (el estadístico BIC aumenta la penalidad en modelos con muchas variables y dicho estadistico aumenta la penalidad en modelos con muchas variables ) quedaron mas bajos en el modelo 2 ajustados por los residuos un AR(3) y un MA(2). Esto se contrasta con la primera analisis comparativo donde el modelo 1 era mejor pero podiamos caer en overfiting dado que el modelo tenia muchos parámetros.

### $Parte 7:$

Realizar una predicción completa para los años 1979 y 1980 de la serie a partir del ajuste que crea conveniente.



In [55]:
Pred_1= forecast(Fit_2.2,h=24) 
tsplot(Pred_1,col="blue", lwd=2, main = "Predicción fit" ,xlab = "Tiempo", ylab = "")

In [53]:
## Construimos los nuevos datos a evaluar el modelo

#Agregamos tiempos para 4 años en el futuro. Notar el -1970 para que quede igual que la que ajustamos.
new_t = seq(1979,1980.917,by=0.083) - 1970
#Lo convierto en serie temporal
new_t = ts(new_t,start=1979,freq=12)

#Predigo sobre los nuevos datos.
new_data = data.frame(t=new_t)
prediccion_1= predict(Fit_2, new_data);

#Convierto el resultado en time series
prediccion_final = ts(prediccion_1,start=1979,freq=12)
prediccion_final


,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
1979,0.4253202,0.4208757,0.3379396,0.2342870,0.1721410,0.1670034,0.1843345,0.1841689,0.1666308,0.1716091,0.2329701,0.3344069
1980,0.4135564,0.4135063,0.3333233,0.2290292,0.1640423,0.1566728,0.1740179,0.1751066,0.1576765,0.1605223,0.2190565,0.3197761


In [50]:
tsplot(flu,xlim=c(1970,1982),ylim=c(0,0.7))
lines(Ajuste_2,col=2)
lines(prediccion_final,col=3)

In [64]:
# Armamos la predicion total con los dos modelos (lm y ARMA)

pred1_hat= ts(Pred_1$fitted,start = 1978, frequency = 12)
print(head(pred1_hat))

pred2_hat = ts(prediccion_final,start = 1978, frequency = 12)
print(head(pred2_hat))

x_hat = pred1_hat + pred2_hat

             Jan         Feb         Mar         Apr         May         Jun
1978  0.05250870  0.09048120 -0.14236451  0.04730136 -0.04097043  0.02788250
           Jan       Feb       Mar       Apr       May       Jun
1978 0.4253202 0.4208757 0.3379396 0.2342870 0.1721410 0.1670034


In [71]:
tsplot(flu,xlim=c(1968,1980),ylim=c(0,1))
lines(x_hat,col=2)

### $Parte 8$: Se dispone además de una serie de temperaturas medias mensuales en EEUU en el archivo `temperaturas.csv`. Analizar:
 * Si hay correlación cruzada entre la serie de muertes por gripe y neumonia y la temperatura observada en cada mes.
 * Si esta correlación se mantiene una vez que se saca el efecto estacional a ambas series. 

In [72]:
temp = ts(read.csv("temperaturas.csv")[,"Value"],start=1968,freq=12)
tsplot(temp)

### Para estudiar la correlacion cruzada entre la serie temp y flu lo que vamos vamos a realizar es lo siguientes:
1- Estudiar la correlacion cruzada con la funcion ccf, cor para el coeficiente de correlacion y estudiar las graficas que se generar con la función lag2.plot con sus respectivos lags.
2- Realizar un modelo basico linear con la serie temp para estudiar los residuos y poder compararlos sin el efecto estacional.
3- Estudiar nuevamente la correlacion cruzada y las graficas para ver como es la correlacion entre las series luego de sacarle el efecto estacional.

In [81]:
## Paso 1

ccf(flu,temp)
ccf_Values_1 = ccf(flu,temp)
ccf_Values_1


Autocorrelations of series ‘X’, by lag

-1.5000 -1.4167 -1.3333 -1.2500 -1.1667 -1.0833 -1.0000 -0.9167 -0.8333 -0.7500 
  0.604   0.559   0.380   0.105  -0.190  -0.437  -0.601  -0.634  -0.466  -0.133 
-0.6667 -0.5833 -0.5000 -0.4167 -0.3333 -0.2500 -0.1667 -0.0833  0.0000  0.0833 
  0.229   0.518   0.661   0.616   0.413   0.101  -0.224  -0.506  -0.672  -0.621 
 0.1667  0.2500  0.3333  0.4167  0.5000  0.5833  0.6667  0.7500  0.8333  0.9167 
 -0.431  -0.125   0.206   0.477   0.612   0.579   0.395   0.127  -0.177  -0.447 
 1.0000  1.0833  1.1667  1.2500  1.3333  1.4167  1.5000 
 -0.573  -0.537  -0.394  -0.144   0.140   0.375   0.500 

In [84]:
lag2.plot (flu, temp, 10)

In [85]:
cor(flu,temp)

[1] -0.6715499

In [74]:
## Paso 2

season = factor(cycle(temp)) 
t = time(temp)-1968

Fit_temp = lm(temp ~ 0 + t + season )
summary(Fit_temp)


Call:
lm(formula = temp ~ 0 + t + season)

Residuals:
   Min     1Q Median     3Q    Max 
-6.034 -1.002  0.261  1.152  5.590 

Coefficients:
         Estimate Std. Error t value Pr(>|t|)    
t         0.02716    0.05062   0.536    0.593    
season1  28.64693    0.60960  46.993   <2e-16 ***
season2  33.75012    0.61137  55.205   <2e-16 ***
season3  41.30968    0.61315  67.373   <2e-16 ***
season4  50.62651    0.61496  82.324   <2e-16 ***
season5  59.83970    0.61680  97.016   <2e-16 ***
season6  68.37652    0.61866 110.524   <2e-16 ***
season7  73.25881    0.62054 118.057   <2e-16 ***
season8  71.62563    0.62244 115.072   <2e-16 ***
season9  64.24155    0.62437 102.890   <2e-16 ***
season10 53.32474    0.62632  85.140   <2e-16 ***
season11 41.23430    0.62829  65.629   <2e-16 ***
season12 32.25567    0.63029  51.176   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.839 on 119 degrees of freedom
Multiple R-squared:  0.999,	Adjus

In [75]:
## Paso 3

resid_Fit_temp = ts(residuals(Fit_temp), start = 1968, freq = 12)
resid_Fit_1 = ts(residuals(Fit_1), start = 1968, freq = 12)
tsplot(resid_Fit_temp)

In [78]:
ccf(resid_Fit_temp,resid_Fit_1)
ccf_Values_2 = ccf(resid_Fit_temp,resid_Fit_1)
ccf_Values_2




Autocorrelations of series ‘X’, by lag

-1.5000 -1.4167 -1.3333 -1.2500 -1.1667 -1.0833 -1.0000 -0.9167 -0.8333 -0.7500 
 -0.054   0.013   0.094   0.063   0.071  -0.003  -0.217  -0.223   0.037   0.119 
-0.6667 -0.5833 -0.5000 -0.4167 -0.3333 -0.2500 -0.1667 -0.0833  0.0000  0.0833 
 -0.039  -0.011  -0.033  -0.012   0.038   0.104   0.119   0.103   0.038  -0.136 
 0.1667  0.2500  0.3333  0.4167  0.5000  0.5833  0.6667  0.7500  0.8333  0.9167 
 -0.176  -0.188  -0.015  -0.042  -0.035  -0.042  -0.020  -0.051  -0.190  -0.050 
 1.0000  1.0833  1.1667  1.2500  1.3333  1.4167  1.5000 
  0.186   0.196   0.031  -0.045   0.027  -0.032   0.038 

In [82]:
lag2.plot (resid_Fit_temp,resid_Fit_1, 10)

In [86]:
cor(resid_Fit_temp,resid_Fit_1)

[1] 0.03793658

### $ Conclusiones: $

Lo que se puede ver es que en el paso 1 donde se estudian las series en su forma normal, existe cierta correlación entre ambas y tanto en el ccf como en las gráficas se pueden especificar con los determinados lags. También si vemos el coeficiente de correlación nos da -0,67.
Por otro lado al sacarle los factores estacionales a las series y realizamos los mismos estudios, se puede observar como la correlación se elimina casi completamente, con un cor de 0.037. Lo mismo se puede ver en los coeficientes de correlación en las gráficas y en el ccf.